In [35]:
#this is my vendor and ground truth data that is coming in 
#we will parse it out into individual elements

class Vendor_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in vendor event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
        
class GroundTruth_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in ground truth event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
        

In [12]:

class Vendor_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in vendor event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]
#        self.runid = data[pointer][68]

        
class GroundTruth_Data(object):
    def __init__(self,data,pointer):
        self.globaleventid = data[pointer][0]
        
        #print 'in ground truth event_data and global_eventid = ',self.globaleventid
        
        self.actor1name = data[pointer][1]
        self.actor2name = data[pointer][2]
        self.actor1mention = data[pointer][3]
        self.actor2mention = data[pointer][4]
        self.actiongeo_fullname = data[pointer][5]
        self.eventcode = data[pointer][6]
        self.guid = data[pointer][7]


'''#import python;
#import aic;

#// spurious - event in vendor set that's not in the gold set
#// unmatched - event in gold set that's not in the vendor set

// For gold records with no events, calculate spurious outside of this call
// For gold records with no vendor records, calculate unmatched outside of this call
// Only make this call if there are gold records AND vendor records for the given id.
// Make this call passing in ALL gold records with the given ID, and ALL vendor records with the given ID.
// Make this call for each ID independently.

// ----- This version only computes the score for the event type field.

export dataset(aic.score.Layouts.L_score) do_computescores2(dataset(aic.score.Layouts.L_in) ds_gold, dataset(aic.score.Layouts.L_in) ds_vendor, utf8 vendor_code, utf8 runid, utf8 gt_cameo_id, utf8 gt_civilunrest_id) := EMBED(Python)
__author__ = "Wei Wang"
__email__ = "tskatom@vt.edu"'''

import sys
import os
from munkres import Munkres
import numpy as np
import re
import csv

import collections

def compute_score(true_event, test_event):
    
	#print 'in compute_score check our gt values',true_event.actor1name
	#print 'in compute_score check our vendor values',test_event.actor1name
	# """
	# Evaluate the score of test event
	# parameters:
	#    true_event: json 
	#    test_event: json
	# """
	################
	# score actor1 #
	################
	# only evaluate the actor1name with true event using partial match
	actor1_score = 0.0
	test_actor1 = test_event.actor1name
	#print 'print vendor actor1',test_actor1    
	# if there are multiple sources, we match the actor1 with 
	# highest score. In Reciproal case, we compare the actor1/2 with 
	# the same set of Reciprocal texts.
	if true_event.actor1name:
		sources = true_event.actor1name.split(";")
	else:
		sources = []
	if true_event.actor1mention:
		new_sources = true_event.actor1mention.split(";")
		for new_source in new_sources:
			sources.append(new_source)

	act_score = 0.
	for source in sources:
		act_score = 3.0 if text_partial_match(source, test_actor1) else 0.
		if act_score > actor1_score:
			actor1_score = act_score
	#print 'actor1 score',act_score
	################
	# score actor2 #
	################
	actor2_score = 0.0
	test_actor2 = test_event.actor2name
	#print 'print vendor actor2',test_actor2
    
	if true_event.actor2name:
		targets = true_event.actor2name.split(";")
	else:
		targets = []
	if true_event.actor2mention:
		new_targets = true_event.actor2mention.split(";")
		for new_target in new_targets:
			targets.append(new_target)

	for target in targets:
		#print 'in targets',target    
		actor_score = 3.0 if text_partial_match(target, test_actor2) else 0
		#print 'actor2 score',actor_score
		if act_score > actor2_score:
			actor2_score = act_score

	##################
	# Score Location #
	##################
	# Match Location with partial match
	location_score = 0.0
	true_location = true_event.actiongeo_fullname.split(";")
	test_location = test_event.actiongeo_fullname.strip()
	#print 'print gt location',true_location    
	#print 'print vendor location',test_location
	if len(true_location) == 0:
		if test_location == "":
			location_score = 3.0
		else:
			location_score = 0.0
	else:
		for loc in true_location:
			tem_score = 3.0 if text_partial_match(loc, test_location) else 0.0
			if tem_score > location_score:
				location_score = tem_score
    
	################
	# score action #
	################
	# compute the action score with two level: root and subtype
	action_score = 0.0
	true_code = true_event.eventcode.split(" ")[0]
	test_code = test_event.eventcode.split(" ")[0]
    
	#print 'checking our event code values for gt -->',true_code
	#print 'checking our test code values for gt -->',test_code
    
	if test_code is None or len(test_code) < 2:
		#print 'in action score fail'        
		action_score = 0.0
	else:
		if true_code[:2] == test_code[:2]:
			#print 'checking action score',true_code[:2],test_code[:2],action_score
			action_score += 2.0
			if len(test_code) > 3 and test_code[2] == true_code[2]:
				action_score += 1.0
    
	###############
	# Date Score ##
	###############
	# since the Date field in the GSR is not normalized, we don't compute the
	# score here
	date_score = 0.0

	final_score = [actor1_score, actor2_score, location_score, action_score, date_score]

	#print 'Final_Score -->',final_score
    
	fields = ["Actor1", "Actor2", "Location", "Action", "Date"]
	return sum(final_score), dict(zip(fields, final_score))

def text_partial_match(true_text, test_text):
	"""
	partial match the true_text and test_text based on following step
	1> transform the text to lowercase 
	2> split the text by space or comma and ignore ''
	3> if any pair of elements from two different text arraies equal to each
	other, return True else False
	"""
	true_text_splits = set([r for r in re.split(r" |,", true_text.lower()) if r != ''])
	test_text_splits = set([r for r in re.split(r" |,", test_text.lower()) if r != ''])
    
	if true_text_splits.intersection(test_text_splits):
		return True
	else:
		return False

def global_match(ground_truth, vendor_events):
	"""
	matching the true events with test events with global optimal score
	"""

    
    
#	if not true_events or not test_events:
#		return 0.0, [], [], []

	#this was original len check, but i think we need to check how many elements are in our inbound list, so changed it     
	#print 'here and length of true_events',len(true_events)    
	#print 'here is the length of our ground truth list-->',len(ground_truth)
	#print 'here is the length of our vendor list-->',len(vendor_events)
    
	gt_guid = []    
	vendor_guid = []
	vendor_score = 0.00
	#print 'records',vendor_events    
	for records, true_event in enumerate(ground_truth):
		#if records == len(ground_truth): break        
		#extract guid
		
		#print records        
		gt_guid.append(ground_truth[records][7])

	#print 'before vendor guid check'       
	#print vendor_events
        
	for counter, records in enumerate(vendor_events):
		#extract guid
		vendor_guid.append(vendor_events[counter][7])
	#print 'all of my gt guids',gt_guid
	#print 'all of my vendor guids',vendor_guid
    
	#lets define our set of guids
	gtguids = list(set(gt_guid))

	#print 'here is my gt set', gtguids
    
	vendorguid = list(set(vendor_guid))
	#print 'here is my vendor set', vendorguid
    
	#let's check how many values are in our vendor set
	guids_to_process = []    
	for guids,groundtruth in enumerate(gtguids):
		#print 'checking',gt         
		if gtguids[guids] in vendorguid:
			guids_to_process.append(gtguids[guids])  
		#else:        
			#print 'no'   
			
	#print 'here are my guids_to_process',guids_to_process 

	#ok, so now lets iterate through out guids
	#print 'before guid count'  
	#score_matrix = []    
	for processing in guids_to_process:
       
		#original code    
		#score_matrix = np.zeros((len(true_events), len(test_events)))
		#score_matrix before switching to iteration of guids
		#score_matrix = np.zeros((len(ground_truth), len(vendor_events)))  
		#before iterating through guids              
		#print 'guid to check',processing   

		#print 'count ground truth',gt_guid.count(processing)
		#gtcountguids = gt_guid.count(processing)         
		#print 'count vendor truth',vendor_guid.count(processing)
		#vendorcountguids = gt_guid.count(processing)
		#I need to filter values based on current processing        
		vendor_1 = filter(lambda vendor_events: vendor_events[7] in processing, vendor_events) 
		gt_1  = filter(lambda ground_truth: ground_truth[7] in processing, ground_truth)        
		#print 'vendors',vendor_1    
		#print 'gt',gt_1           
        
		score_matrix = np.zeros((len(gt_1),len(vendor_1)))  
		#print 'score matrix -->',score_matrix 
        
		#print 'score matrix -->',score_matrix    
    
		score_detail = {}
		for i, true_event in enumerate(gt_1):
		#if i%500==0:
				#print 'outer loop divisible by 500 counter is @',i
		#print 'loop counter',i
			for j, test_event in enumerate(vendor_1):
				true_events = GroundTruth_Data(gt_1,i)

			#if j%500==0:
				#print 'inner loop divisible by 500 counter is @',j            
    
			#print 'check true event value', true_events.actor1name
    
				test_events = Vendor_Data(vendor_1,j)    
			#print 'check vendor event value', test_events.actor1name
			#print 'loop counter',j
            
				score, score_list = compute_score(true_events, test_events)
                
				if score > 0:
					#print 'here is the score for guid:',processing,score            
					score_matrix[i, j] = score
					#print 'here is the current score matrix',score_matrix[i,j]            
					score_detail[(i,j)] = {"score": score, "detail": score_list, "pair": [true_event, test_event]}
					#print 'here is the current score detail',score_detail[(i,j)] 
					#if processing == '0070cb46-4a4e-4964-8904-6283bd1c34e0' and score >0:
							#print 'processing',processing,score,score_list,test_event 
 
				#else:
					#print 'No Score for this guid',processing                                
		#print 'I am done processing current guid'
        
		cost_matrix = 10000 - score_matrix
		#print 'here'    
		m = Munkres()

		#print 'here is cost_matrix',cost_matrix    
    
		indexes = m.compute(cost_matrix.tolist())
		#print 'here are my indexes',indexes
		total_score = sum(score_matrix[zip(*indexes)])
		event_pairs = []
		#going to track total score        
		vendor_score += total_score
		#if processing == '6b4f7c60-1141-4259-b551-15ebb0b88b84':
			#print 'true_events',true_event
		print 'The total score for guid %s is: %s'%(processing,total_score)        
		unmatched_events = []
		spurious_events = []
		#matched_true_ids, matched_test_ids = zip(*indexes)
		#print matched_true_ids
		#print 'score detail',score_detail         
	return vendor_score    
'''     
		matched_true_ids, matched_test_ids = zip(*indexes)

		print 'checking my id matches for gt and vendor -->',matched_true_ids,matched_test_ids    
    
		for i, true_event in enumerate(gt_1):
			if i not in matched_true_ids:
				unmatched_events.append(true_event)
		for j, test_event in enumerate(vendor_1):
			if j not in matched_test_ids:
				spurious_events.append(test_event)

		for pair in indexes:
			if score_detail[pair]["score"] != 0.0:
				event_pairs.append(score_detail[pair])
			else:
			#print'am I here'            
				unmatched_events.append(gt_1[pair[0]])
				spurious_events.append(vendor_1[pair[1]])
    
		#return total_score, event_pairs, unmatched_events, spurious_events
''' 
# --------------------just a line break for better reading------------------
# start of processing
# let's define our data to test

#vendor_data = '/Users/Pete/Customers/scoring_gt16_2.csv'
#vendor_data='/Users/Pete/Customers/scoring_gt16_subset1.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt6.score.csv'
#vendor_data='/Users/Pete/July25FlattenclusterGT7e.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt16.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt10.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gt14.score.csv'
#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/GT7.scoring.csv'
#vendor_data= '/Users/Pete/Customers/gt7selectcolumns.csv'
#vendor_data='/Users/Pete/Customers/OneGuidGT7SelectColumns.csv'
#vendor_data='/Users/Pete/July26GT7GTMatches.csv'
#vendor_data='/Users/Pete/July26GT9Matches.csv'
#vendor_data='/Users/Pete/July26GT17Matches.csv'
#vendor_data='/Users/Pete/Customers/gt7_runidmay2016.csv'
#vendor_data='/Users/Pete/July26gt10_matches.csv'
#vendor_data='/Users/Pete/July26gt16_matches.csv'
#vendor_data='/Users/Pete/July26g6_matches.csv'
#vendor_data='/Users/Pete/July25FlattenclusterGT7.csv'
#vendor_data='/Users/Pete/Customers/AllVendorsMerged.csv'
vendor_data='/Users/Pete/Customers/gt5gtmatch.csv'
#vendor_processing = raw_input('Which vendor are we processing? ')


vendor_processing = 'ALL'

#vendor_data='/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/gfcluster1.score.csv'

#vendor_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/tamr.cluster2tet.csv'
with open(vendor_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    dsvendor = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists

#gt_data = '/Users/Pete/Customers/scoring_groundtruthcombined_2.csv'
#gt_data = '/Users/Pete/Customers/scoring_groundtruthcombined_subset1.csv'
gt_data = '/Users/Pete/Customers/2016.Feb Greenfield/July20ScoringRun/groundtruthcombined.score.csv'

#gt_data = '/Users/Pete/Customers/gt.matchtest1'
with open(gt_data, 'rU') as f:  #opens PW file
    reader = csv.reader(f)
    dsgold = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
   
    
''' 
file structure
globaleventid
actor1name
actor2name
actor1mention
actor2mention
actiongeo_fullname
eventcode
'''

#dsgold = [['12345','Guido','Rossum','France',' ',' ','015','1'],['0036','Guido','Rossum','France','Place',' ','013','2']]

#dsgold = [['12345','Guido','Rossum','France',' ',' ','0151']]

#dsvendor = [['12345','Pete',' ','France',' ',' ','015','1'],['0036','Guido','Rossum','France',' ',' ','013','2'],['12345','Guido','Rossum','France',' ',' ','0151','4']]

unmatched = 0
matched = 0
spurious = 0
total_score = 0. 
total_actor1_score = 0.
total_actor2_score = 0.
total_location_score = 0.
total_action_score = 0.
total_date_score = 0.

# PJD this comment is from original source -->This seems silly, but seems to be necessary - HPCC lists passed in have no length
# PJD don't think we need this 
'''
dsgold = []
for item in ds_gold:
	dsgold.append(item)
dsvendor = []
for item in ds_vendor:
	dsvendor.append(item)
'''

#print 'I am right before global_match call'

#score, event_pairs, unmatched_events, spurious_events = 

total_score = global_match(dsgold, dsvendor)

print 'The total score for vendor %s is: %s'%(vendor_processing,total_score) 

#print 'here are my event_pairs',event_pairs
#print 'here are my unmatched_events,spurious_events',unmatched_events
#print 'here are my spurious_events',spurious_events



'''
matched_event_pairs = []
total_score += score
#print 'total_score at end',total_score
#print 'event_pairs',event_pairs

if event_pairs:
	for event_pair in event_pairs:
		total_actor1_score += event_pair["detail"]["Actor1"]
		print 'total_actor1_score',total_actor1_score
		total_actor2_score += event_pair["detail"]["Actor2"]
		print 'total_actor2_score',total_actor2_score
		total_location_score += event_pair["detail"]["Location"]
		print 'total_location_score',total_location_score
		total_action_score += event_pair["detail"]["Action"]
		print 'total action score',total_action_score
		total_date_score += event_pair["detail"]["Date"]
		print 'total date score',total_date_score
        

		matched_event_pairs.append((event_pair["pair"][0][0], event_pair["pair"][1][0], event_pair["pair"][0][6], event_pair["detail"]["Actor1"], event_pair["detail"]["Actor2"], event_pair["detail"]["Location"], event_pair["detail"]["Action"], event_pair["detail"]["Date"]))
      
		#matched_event_pairs.append((event_pair["pair"][0].globaleventid, event_pair["pair"][1].globaleventid, event_pair["pair"][0].eventcode, event_pair["detail"]["Actor1"], event_pair["detail"]["Actor2"], event_pair["detail"]["Location"], event_pair["detail"]["Action"], event_pair["detail"]["Date"]))

matched += len(event_pairs)
unmatched += len(unmatched_events)
#print 'unmatched events length',unmatched
spurious += len(spurious_events)
#print 'spurious events length',spurious

unmatch_events = []
  
if unmatched_events:
	for unmatched_event in unmatched_events:
#append global eventid and eventcode        
		unmatch_events.append((unmatched_event[0], unmatched_event[6]))
spur_events = []
if spurious_events:
	for spur_event in spurious_events:
		spur_events.append((spur_event[0], spur_event[6]))

#result = [(u'event_type', vendor_code, runid, gt_cameo_id, gt_civilunrest_id, unmatched, matched, spurious, total_score, total_actor1_score, total_actor2_score, total_location_score, total_action_score, total_date_score, matched_event_pairs, unmatch_events, spur_events)]
'''

The total score for guid 7e1e8bc4-f58f-4043-87d2-81b5a4293e67 is: 0.0
The total score for guid 82af1c43-9573-44d6-8a28-296f481a6006 is: 0.0
The total score for guid 965e0493-31f0-4e8f-88c4-75d057b3eebf is: 0.0
The total score for guid 0e996f08-5506-42bc-afc2-22c03f0f28c5 is: 0.0
The total score for guid 630d1492-91c7-411c-91b1-2542e1bfa368 is: 0.0
The total score for guid 2d38c42b-33f5-4153-97a4-73f45ff99c86 is: 0.0
The total score for guid a634c9bf-bcbe-408b-a497-796441b65294 is: 0.0
The total score for guid cb01c2e7-5cc0-4cf9-b19d-c1de6d8b6cf3 is: 0.0
The total score for guid 66b1144b-1d87-4db1-b436-5087c604baaa is: 0.0
The total score for guid b4b71156-8603-4553-961c-835b943f865d is: 0.0
The total score for guid 890000ce-2b9b-49d9-9023-967d57f8a65f is: 0.0
The total score for guid 0c0f1c7d-bb89-4c0c-9262-ff0f98b69e0d is: 0.0
The total score for guid d1d52912-f38b-4c18-88df-14e6e0056044 is: 0.0
The total score for guid 6992fa66-e04d-470c-92e4-cc765ee6fa81 is: 0.0
The total score for 

'\nmatched_event_pairs = []\ntotal_score += score\n#print \'total_score at end\',total_score\n#print \'event_pairs\',event_pairs\n\nif event_pairs:\n\tfor event_pair in event_pairs:\n\t\ttotal_actor1_score += event_pair["detail"]["Actor1"]\n\t\tprint \'total_actor1_score\',total_actor1_score\n\t\ttotal_actor2_score += event_pair["detail"]["Actor2"]\n\t\tprint \'total_actor2_score\',total_actor2_score\n\t\ttotal_location_score += event_pair["detail"]["Location"]\n\t\tprint \'total_location_score\',total_location_score\n\t\ttotal_action_score += event_pair["detail"]["Action"]\n\t\tprint \'total action score\',total_action_score\n\t\ttotal_date_score += event_pair["detail"]["Date"]\n\t\tprint \'total date score\',total_date_score\n        \n\n\t\tmatched_event_pairs.append((event_pair["pair"][0][0], event_pair["pair"][1][0], event_pair["pair"][0][6], event_pair["detail"]["Actor1"], event_pair["detail"]["Actor2"], event_pair["detail"]["Location"], event_pair["detail"]["Action"], event_pair